# AoC 2024 Day 12
https://adventofcode.com/2024/day/12

In [2]:
from collections import defaultdict, deque

In [3]:
with open('data/day12.txt') as f:
    data = f.read().split()

In [4]:
A = 'A'  # across 
D = 'D'  # down
ADJACENTS = ((1,0), (0,1), (-1,0), (0, -1))

def in_box(i, j, rows, cols):
    return 0 <= i < rows and 0 <= j < cols

def get_val_suffixed(val, letter_map):
    suffix = 1
    new_val = val + str(suffix)
    while new_val in letter_map:
        suffix += 1
        new_val = val + str(suffix)
    return new_val

def get_letter_map(data) -> dict[list[tuple[int, int]]]:
    letter_map = defaultdict(set)
    mapped = set()
    rows = len(data)
    cols = len(data[0])
    for i, row in enumerate(data):
        for j, val in enumerate(row):
            if (i,j) in mapped:
                continue
            val_suffixed = get_val_suffixed(val, letter_map)
            queue = deque()
            seen = set()
            queue.append((i,j))
            seen.add((i,j))
            while queue:
                ci, cj = queue.popleft()
                if data[ci][cj] == val:
                    letter_map[val_suffixed].add((ci, cj))
                    mapped.add((ci, cj))
                    new = [(ci+di, cj+dj) for di, dj in ADJACENTS 
                                  if in_box(ci+di, cj+dj, rows, cols) 
                                  and (ci+di, cj+dj) not in mapped
                                  and (ci+di, cj+dj) not in seen
                                  and data[ci+di][cj+dj] == val]
                    seen.update(new)
                    queue.extend(new)
    return letter_map

def get_perimeter(positions: list[tuple[int, int]]) -> set:
    all_perimeter_blocks = set()
    for i, j in positions:
        perimeter_blocks = set([(i, j, A), (i, j, D), (i+1, j, A), (i, j+1, D)])
        all_perimeter_blocks = all_perimeter_blocks.symmetric_difference(perimeter_blocks)
    return all_perimeter_blocks

def get_price(data):
    letter_map = get_letter_map(data)
    price = 0
    for letter, positions in letter_map.items():
        price += len(positions)*len(get_perimeter(positions))
    return price

get_price(data)

1449902

In [11]:
def get_number_of_perimeter_sides(positions: list[tuple[int, int]]) -> int:
    perimeter_blocks = get_perimeter(positions)
    across_fences = sorted([f for f in perimeter_blocks if f[2] == A])
    down_fences = sorted([f for f in perimeter_blocks if f[2] == D], key=lambda x: (x[1], x[0]))
    # print(across_fences)
    # print(down_fences)
    sides = 1
    for i in range(1, len(across_fences)):
        curr = across_fences[i]
        prev = across_fences[i-1]
        # print(prev)
        if not are_neighbours(curr, prev, perimeter_blocks):
            sides += 1
            # print('split')
    sides += 1
    for i in range(1, len(down_fences)):
        curr = down_fences[i]
        prev = down_fences[i-1]
        # print(prev)
        if not are_neighbours(curr, prev, perimeter_blocks):
            sides += 1
            # print('split')
    return sides

def are_neighbours(f1, f2, perimeter_blocks):
    if f1 > f2:
        f1, f2 = f2, f1
    if f1[2] == f2[2] == A:
        return f1[0] == f2[0] and abs(f1[1] - f2[1]) == 1 and (f2[0]-1, f2[1], D) not in perimeter_blocks
    elif f1[2] == f2[2] == D:
        return f1[1] == f2[1] and abs(f1[0] - f2[0]) == 1 and (f2[0], f2[1]-1, A) not in perimeter_blocks
    else:
        return False
        

def get_discounted_price(data):
    letter_map = get_letter_map(data)
    price = 0
    for letter, positions in letter_map.items():
        # if letter != 'K3':
            # continue
        n = get_number_of_perimeter_sides(positions)
        # print(letter, len(positions), n)
        price += len(positions)*n
    return price
get_discounted_price(data)

908042